In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [3]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [4]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer  # Нужно для использования IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, PowerTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import TransformedTargetRegressor

### Область работы 2 (выполнение лучшей модели)

In [6]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [7]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [8]:
# Блок обучения модели

In [9]:
df_test=pd.read_csv(path_test)
df=pd.read_csv(path_train)

df = df[df['Length'] != 46]

In [10]:
cat_features = ['Age', 'Sex']
axis_features = ['Length', 'Girth', 'Height', 'BCS']

In [11]:
X = df.drop('Weight', axis=1)
y = df['Weight']

In [12]:
axis_transformer = Pipeline(steps=[
    # ('missing_num', IterativeImputer(missing_values=np.nan, max_iter=20)),
    ('polynom', PolynomialFeatures(2, include_bias=False))
    # ('scaler', MinMaxScaler())
])

# num = Pipeline(steps=[
#     ('missing_num', IterativeImputer(missing_values=np.nan, max_iter=20)),
#     # ('scaler', MinMaxScaler()) 
# ])

cat = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='most_frequent')), 
    # ('imputer', IterativeImputer(missing_values=np.nan, max_iter=20)),
    ('onehot', OneHotEncoder(sparse_output=False))
])


CT = ColumnTransformer([
    ("axis_transformer", axis_transformer, axis_features),
    # ("num", num, num_features),
    ("cat", cat, cat_features),
    # ("drop_columns", "drop", drop_features)  # Указание на удаление колонок
]).set_output(transform='pandas')

display(CT)
ct = CT.fit_transform(X)
# ct

ColumnTransformer(transformers=[('axis_transformer',
                                 Pipeline(steps=[('polynom',
                                                  PolynomialFeatures(include_bias=False))]),
                                 ['Length', 'Girth', 'Height', 'BCS']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(sparse_output=False))]),
                                 ['Age', 'Sex'])])

In [13]:
pipeline = Pipeline(steps=[
    ('preproc', CT),
    ('estimator', RandomForestRegressor(criterion='squared_error', n_estimators=1000, oob_score = True, n_jobs=-1))
])

In [14]:
rf = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=PowerTransformer(method='box-cox')
)

In [15]:
rf.fit(X, y)

TransformedTargetRegressor(regressor=Pipeline(steps=[('preproc',
                                                      ColumnTransformer(transformers=[('axis_transformer',
                                                                                       Pipeline(steps=[('polynom',
                                                                                                        PolynomialFeatures(include_bias=False))]),
                                                                                       ['Length',
                                                                                        'Girth',
                                                                                        'Height',
                                                                                        'BCS']),
                                                                                      ('cat',
                                                                                       Pipeline(steps=[('onehot',
                                                                                                        OneHotEncoder(sparse_output=False))]),
                                                                                       ['Age',
                                                                                        'Sex'])])),
                                                     ('estimator',
                                                      RandomForestRegressor(n_estimators=1000,
                                                                            n_jobs=-1,
                                                                            oob_score=True))]),
                           transformer=PowerTransformer(method='box-cox'))

In [16]:
# Блок предсказания с использованием тестового набора

In [17]:
y_predict = rf.predict(df_test)

In [18]:
# Вектора предсказанных значений  y_predict полученый на основане тестового набора
y_predict

array([142.95640474, 145.51121089, 163.94664127, 154.43980004,
       176.02965314, 122.71573273, 158.45816021, 166.23874351,
       136.11067087, 179.91398442, 129.9640879 , 154.94743201,
       193.17724833, 132.54587155, 128.16832796, 172.34588362,
       162.32426672, 154.40569364, 142.5328609 , 147.95110493,
       155.63453349, 155.54861158, 145.80626328, 156.19263182,
       175.96089736, 167.46521727, 152.53941683,  87.70633429,
       159.45629979, 164.42343317, 157.5903115 , 154.00670625,
       191.95146638, 193.09370094, 128.33856957, 159.92365084,
       172.54437111, 174.73399456, 123.75807125, 177.85676009,
       161.10524605, 158.6708551 , 172.10702891, 138.06582713,
       167.00107895, 171.4400197 , 134.97836146, 100.00494517,
       199.74854027, 159.93835142, 156.89593328, 145.8448825 ,
       182.60118214, 155.80865567, 181.21483563, 174.91392868,
       121.52091129, 164.21672026, 146.33068439, 139.48036749,
       178.95703996,  94.4048204 , 134.66021056, 169.07